In [ ]:
#imports
import numpy as np
from glob import glob
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt                        
%matplotlib inline
import os
import datetime
import time
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
#load data
dataPath="/media/amrelrasad/LENOVO/Backup_Linux/Documents/Pytorch_Udacity/deep-learning-v2-pytorch/project-dog-classification/dogImages"
#define data transforms
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
valid_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# data
train_data = datasets.ImageFolder(dataPath + '/train', transform=train_transforms)
valid_data = datasets.ImageFolder(dataPath+ '/valid', transform=valid_transforms)
test_data = datasets.ImageFolder(dataPath+ '/test', transform=test_transforms)

# loaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=64)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [ ]:
#model parameters
net_dic={0:"alexnet",
        1:"vgg11",
        2:"vgg13",
        3:"vgg16",
        4:"vgg19",
        5:"vgg11_bn",
        6:"vgg13_bn",
        7:"vgg16_bn",
        8:"vgg19_bn",
        9:"resnet18",
        10:"resnet34",
        11:"resnet50",
        12:"resnet101",
        13:"resnet152",
        14:"densenet121",
        15:"densenet169",
        16:"densenet201",
        17:"densenet161",
        18:"inception_v3",
}
FC=[9216,25088,25088,25088,25088,25088,25088,25088,25088,512,512,2048,2048,2048,1024,1664,1920,2208,2048]
net_index=11
selected_net=net_dic[net_index]
output_classes=len(train_data.class_to_idx)
full_train=True
net_pretrained=True
retrain=True
learning_rate=0.00000001
# enter model path and file name where you want to load or save your new model
model_path="image_model.pth"


In [ ]:
print(output_classes)
print(selected_net)

In [ ]:
def test(loader, model,criterion, device):
    test_loss = 0.0
    test_count=0.0
    test_size=0.0
    test_accuracy = 0.0
    model.eval()
    with torch.no_grad():
        for inputs, labels in loader:
            test_size+=len(labels)
            inputs, labels = inputs.to(device), labels.to(device)
            logps = model.forward(inputs)
            batch_loss = criterion(logps, labels)
            test_loss += batch_loss.item()
            # Calculate accuracy
            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            test_count+=torch.sum(equals.type(torch.FloatTensor))
        test_accuracy=test_count/test_size
        test_loss=test_loss/len(loader)
    return test_accuracy,test_loss,test_count

In [ ]:
def test_report(model, loader,device):
    test_count=0.0
    test_size=0
    test_accuracy = 0.0
    predictions=[]
    true_labels=[]
    model.eval()
    with torch.no_grad():
        for inputs,labels in loader:
            test_size+=len(inputs)
            inputs=inputs.to(device)
            labels=labels.to(device)
            Z = model.forward(inputs)
            ps = torch.exp(Z)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            test_count+=torch.sum(equals.type(torch.FloatTensor))
            true_labels+=(labels.squeeze().tolist())
            predictions+=(top_class.squeeze().tolist())
        test_accuracy=test_count/test_size
        print(classification_report(true_labels, predictions))
            
    return test_accuracy,test_count

In [ ]:
def train(epochs, trainloader,validloader, model, optimizer, criterion, device, model_path_name,valid_count_max):

    start=time.time()
    print (datetime.datetime.now())
    steps = 0

    print_every = 200
    #test_acc_max = 0.8*len(testloader) # track change in validation loss
    for epoch in range(epochs):
        step=0
        train_loss = 0.0
        model.train()
        for inputs, labels in trainloader:
            step += 1
            if(step%print_every==0):
                print(f"Epoch {epoch+1}/{epochs}..step {step}/{len(trainloader)}.."
                      f"time so far {(time.time()-start)/60:.3f} mins")
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            logps = model.forward(inputs)
            loss = criterion(logps, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            # Calculate accuracy
            ps = torch.exp(logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
        train_loss=train_loss/len(trainloader)
        valid_accuracy, valid_loss,valid_count=test(validloader,model,criterion,device)
        print (datetime.datetime.now())
        print(f"Epoch {epoch+1}/{epochs}.. "
              f"Train loss: {train_loss:.3f}.. "
              f"Valid loss: {valid_loss:.3f}.. "
              f"Valid accuracy: {valid_accuracy:.5f}.."
              f"Time Elspased so far: {time.time()-start:.3f} seconds")
        train_loss = 0
        train_accuracy=0
        if valid_count>valid_count_max:
            print(f"*************New Model Saved with correct predictions of:{valid_count:.5f}***************")
            torch.save(model, model_path_name)
            valid_count_max= valid_count
    return model,valid_count_max

In [ ]:
# Model instainstiation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if(retrain):
    model = torch.load(model_path)
else:
    model=eval("models."+selected_net+"(pretrained=net_pretrained)")
    torch.save(model, model_path_name)
    if(not full_train):
        for par in  model.parameters():
            param.requires_grad=False
    #change FC classifier
    model.fc = nn.Sequential(nn.Linear(FC[net_index], 1024),
                                     nn.ReLU(),
                                     nn.Linear(1024, 256),
                                     nn.ReLU(),
                                     #nn.Dropout(0.2), #optional
                                     nn.Linear(256, output_classes),
                                     nn.LogSoftmax(dim=1)) 
    

criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

model.to(device);

In [ ]:
print(device)
print(model)

In [ ]:
# test model intially over validation set
valid_acc_max,valid_loss,valid_count_max=test(validloader,model,criterion,device)
#valid_acc_max=valid_accuracy/len(validloader)

In [ ]:
valid_acc_max,valid_loss,valid_count_max

In [ ]:
#train model
epochs=3
model,valid_count_max = train(epochs, trainloader,validloader, model, optimizer,criterion,device,model_path,valid_count_max) 


In [ ]:
model = torch.load(model_path)

In [ ]:
#test model

test_accuracy,test_loss,test_count=test(testloader,model,criterion,device)
print(f"Test loss: {test_loss:.3f}.. "
    f"Test accuracy: {test_accuracy:.5f}.."
    f"Test count: {test_count:.5f}.. ")

In [ ]:
loader_dic={"train":trainloader,"valid":validloader,"test":testloader}
#loader_dic={"valid":validloader,"test":testloader}
for i in loader_dic:
    loader=loader_dic[i]
    print(i+" data set")
    test_report(model,loader,device)
    print("*"*50)

In [ ]:
for i in net_dic.keys():
    model=eval("models."+net_dic[i]+"(pretrained=net_pretrained)")
    print("*"*60)
    print(net_dic[i])
    print("*"*60)
    print (model)
    print("*"*60)

In [ ]:
FC=[9216,25088,25088,25088,25088,25088,25088,25088,25088,512,512,2048,2048,2048,1024,1664,1920,2208,2048]
len(FC)